In [1]:
# !pip install wikipedia-api mwparserfromhell

## Initialize Wikipedia API
Setup the Wikipedia API client for Arabic language

In [2]:
import wikipediaapi
import json
import os

wiki = wikipediaapi.Wikipedia(
    user_agent='AgriKnowledgeBot/1.0 (Education Purpose)',
    language='ar',
    extract_format=wikipediaapi.ExtractFormat.WIKI
)

## Get All Categories and Subcategories
This function recursively collects all categories and subcategories from a seed category, removing duplicates

In [3]:
def get_all_categories(seed_category, max_depth=2):
    """
    Get all categories and subcategories from a seed category.
    Returns a set of unique category names.
    """
    categories = set()
    visited = set()
    
    def recurse(category_name, current_depth):
        if category_name in visited or current_depth >= max_depth:
            return
        
        visited.add(category_name)
        cat = wiki.page(category_name)
        
        if not cat.exists():
            print(f"⚠️ Category not found: {category_name}")
            return
        
        # Add current category to the set
        categories.add(category_name)
        
        # Get subcategories (namespace 14)
        for title, page in cat.categorymembers.items():
            if page.ns == 14:  # Category namespace
                recurse(title, current_depth + 1)
    
    recurse(seed_category, 0)
    return categories


# Test with one seed category
test_categories = get_all_categories("تصنيف:الزراعة في مصر")
print(f"Found {len(test_categories)} unique categories")
for cat in sorted(test_categories)[:10]:
    print(f"  - {cat}")

Found 10 unique categories
  - تصنيف:التبغ في مصر
  - تصنيف:الزراعة في مصر
  - تصنيف:رعاية الحيوان في مصر
  - تصنيف:ري في مصر
  - تصنيف:سلالات حيوانية نشأت في مصر
  - تصنيف:شركات زراعة في مصر
  - تصنيف:فلاحون مصريون
  - تصنيف:متاحف زراعة في مصر
  - تصنيف:محاصيل منشأها مصر
  - تصنيف:منظمات زراعية مقرها في مصر


## Get Pages from Categories
Extract all pages (ns == 0) from the collected categories

In [4]:
list(test_categories)

['تصنيف:فلاحون مصريون',
 'تصنيف:الزراعة في مصر',
 'تصنيف:محاصيل منشأها مصر',
 'تصنيف:رعاية الحيوان في مصر',
 'تصنيف:التبغ في مصر',
 'تصنيف:منظمات زراعية مقرها في مصر',
 'تصنيف:سلالات حيوانية نشأت في مصر',
 'تصنيف:متاحف زراعة في مصر',
 'تصنيف:ري في مصر',
 'تصنيف:شركات زراعة في مصر']

In [6]:
from tqdm import tqdm
import time

def get_pages_from_categories(categories_set):
    """
    Get all pages (ns == 0) from a set of categories.
    Returns a set of unique page titles.
    """
    pages = set()
    
    # Use tqdm to show progress
    for category_name in tqdm(categories_set, desc="Processing categories"):
        time.sleep(0.1)  # small delay to avoid overloading wiki
        cat = wiki.page(category_name)
        if not cat.exists():
            continue
        
        # Get pages in this category (namespace 0)
        category_pages = {title for title, page in cat.categorymembers.items() if page.ns == 0}
        pages.update(category_pages)
    
    return pages


# Test with our test categories
test_pages = get_pages_from_categories(test_categories)
print(f"\nFound total {len(test_pages)} unique pages")
for page in sorted(test_pages)[:5]:
    print(f"  - {page}")


Processing categories: 100%|██████████| 10/10 [00:05<00:00,  1.86it/s]


Found total 61 unique pages
  - إعدام الخنازير في مصر
  - البطاطس في مصر
  - البنك الزراعي المصري
  - التدخين في مصر
  - الثروة الحيوانية في مصر


a very related cat to egypt you can use 
```
[
 'تصنيف:الزراعة في مصر',
 'تصنيف:رعاية الحيوان في مصر',
 'تصنيف:ري في مصر',
 'تصنيف:سلالات حيوانية نشأت في مصر',
 'تصنيف:شركات زراعة في مصر',
 'تصنيف:فلاحون مصريون',
 'تصنيف:متاحف زراعة في مصر',
 'تصنيف:محاصيل منشأها مصر']
 ```

## Process All Agriculture Categories
Process all seed categories and collect unique pages

In [7]:
# this is filtered version that specific to our needs
seed_categories = [
    'تصنيف:آفات زراعية', 'تصنيف:آلات زراعية', 'تصنيف:أدوات زراعية', 'تصنيف:أرض زراعية', 'تصنيف:أسمدة', 
    'تصنيف:أشجار مثمرة', 'تصنيف:أعشاب طبية', 'تصنيف:أعلاف', 'تصنيف:أعمال عن الزراعة', 
    'تصنيف:أمراض النباتات', 'تصنيف:إصلاح الأراضي', 'تصنيف:استصلاح الأراضي', 'تصنيف:اقتصاد زراعي', 
    'تصنيف:الزراعة حسب المنطقة', 'تصنيف:الزراعة حسب النوع', 
    'تصنيف:الزراعة في الثقافة الشعبية', 'تصنيف:الزراعة في مصر', 'تصنيف:الزراعة في الوطن العربي', 
    'تصنيف:الزراعة والبيئة', 'تصنيف:الصحة والسلامة الزراعية', 
    'تصنيف:بذرة زراعة', 'تصنيف:بستنة', 'تصنيف:بستنة وحدائق', 'تصنيف:بقوليات', 
    'تصنيف:تربة', 'تصنيف:تربية الحيوان', 'تصنيف:تسميد', 'تصنيف:تعليم زراعي', 
    'تصنيف:تغذية النبات', 'تصنيف:تقنيات زراعية', 'تصنيف:ثروة حيوانية', 'تصنيف:حبوب', 'تصنيف:حشرات زراعية', 
    'تصنيف:حصاد', 'تصنيف:خضراوات', 'تصنيف:دواجن', 'تصنيف:ري', 'تصنيف:ري حديث', 
    'تصنيف:ري قديم', 'تصنيف:زراعة', 'تصنيف:زراعة استوائية', 'تصنيف:زراعة حسب الثقافة', 
    'تصنيف:زراعة حسب المحصول', 'تصنيف:زراعة حضرية', 'تصنيف:زراعة عضوية', 'تصنيف:زراعة في الماء', 'تصنيف:زراعة مائية', 
    'تصنيف:زراعة مستدامة', 'تصنيف:سياسات زراعية', 'تصنيف:ضمان المزارع', 
    'تصنيف:طب بيطري', 'تصنيف:علم أمراض النبات', 'تصنيف:علم الإنتاج النباتي', 'تصنيف:علم البيئة الزراعي', 
    'تصنيف:علم التربة', 'تصنيف:علم زراعي', 'تصنيف:عمل زراعي', 'تصنيف:عمليات زراعية', 'تصنيف:فاكهة', 
    'تصنيف:فلاحون', 'تصنيف:كيماويات زراعية', 
    'تصنيف:ماشية', 'تصنيف:مبان زراعية', 'تصنيف:مبيدات', 'تصنيف:مبيدات حشرية', 
    'تصنيف:مجتمع ريفي', 'تصنيف:محاصيل', 'تصنيف:محاصيل زراعية', 'تصنيف:محطات زراعية', 'تصنيف:مزارع سمكية', 
    'تصنيف:مصطلحات زراعية', 'تصنيف:منتجات زراعية', 'تصنيف:منشآت زراعية', 'تصنيف:منظمات زراعية', 
    'تصنيف:مهن زراعية', 'تصنيف:مواش', 'تصنيف:نباتات اقتصادية', 'تصنيف:نباتات زيتية', 'تصنيف:نباتات زينة', 
    'تصنيف:نباتات طفيلية', 'تصنيف:نباتات مستأنسة', 'تصنيف:هندسة زراعية'
]


# all_categories = set()
# all_pages = set()

# for seed in seed_categories:
#     print(f"\n🔍 Processing seed category: {seed}")
    
#     # Get all categories from this seed
#     categories = get_all_categories(seed, max_depth=2)
#     print(f"  ✓ Found {len(categories)} categories")
    
#     # Get all pages from these categories
#     pages = get_pages_from_categories(categories)
#     print(f"  ✓ Found {len(pages)} pages")
    
#     # Add to overall sets (automatic deduplication)
#     all_categories.update(categories)
#     all_pages.update(pages)
    
#     print(f"  📊 Running totals: {len(all_categories)} categories, {len(all_pages)} pages")

# print(f"\n✅ Final results:")
# print(f"  Total unique categories: {len(all_categories)}")
# print(f"  Total unique pages: {len(all_pages)}")


pages = get_pages_from_categories(seed_categories)

print(f"  Total unique pages: {len(pages)}")

Processing categories: 100%|██████████| 82/82 [00:45<00:00,  1.82it/s]

  Total unique pages: 2308


## Save Results
Save the unique categories and pages to JSON files

In [8]:
# Create output directory
out_dir = "output"
os.makedirs(out_dir, exist_ok=True)

# Save pages
pages_path = f"{out_dir}/agriculture_pages_optimized.json"
with open(pages_path, "w", encoding="utf-8") as f:
    json.dump(sorted(list(pages)), f, ensure_ascii=False, indent=2)
print(f"✓ Saved {len(pages)} pages to: {pages_path}")


✓ Saved 2308 pages to: output/agriculture_pages_optimized.json


In [ ]:
# # Create output directory
# out_dir = "output"
# os.makedirs(out_dir, exist_ok=True)

# # Combine data into a single dict
# output_data = {
#     "categories": sorted(list(all_categories)),
#     "pages": sorted(list(all_pages))
# }

# # Save combined JSON
# combined_path = f"{out_dir}/agriculture_data_optimized.json"
# with open(combined_path, "w", encoding="utf-8") as f:
#     json.dump(output_data, f, ensure_ascii=False, indent=2)

# print(f"✓ Saved {len(all_categories)} categories and {len(all_pages)} pages to: {combined_path}")
